In [8]:
import sqlite3
import numpy as np
import pandas as pd
from pandas import HDFStore

import pubchempy as pcp
import json
from urllib.request import urlopen
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools


import os as os
from os import system, path, remove
import glob
import time
import shutil

import platform
host = platform.node()

#file modes for chmod commands
mod = 0o755

btime = time.strftime("%Y-%m-%d %H:%M")
#print(btime)


#hostflag = 0
if (host == 'boron' or 'sausage'):
    home = '/home/huffman/work/matinsy/'
    dbfile = home+'db/cheminventory.db'
    webhtmldir = './'
    webmsdsdir = webhtmldir+'msds/'
    websafetyplansdir = webhtmldir+'Lab_Specific_Hygiene_Plans/'
    htmldir = '/home/huffman/public_html/sdsweb/'
    safetyplansdir = htmldir+'Lab_Specific_Hygiene_Plans/'
    safetyplansnoplan = './noplans.html'
    msdsdir = htmldir+'msds/'
    roomfile = home+'etc/allrooms.dat'
    cheminfodir = htmldir+'cheminfo/'
    cheminfodata = home+'cheminfodata/'
    
elif host == 'msds.wcu.edu':
    home = '/wwbintz/'
    dbfile = home+'/matinsy/db/cheminventory.db'
    htmldir = home+'public_html/'
    webhtmldir = './'
    safetyplansdir = htmldir+'Lab_Specific_Hygiene_Plans/'
    safetyplansnoplan = './noplans.html'
    msdsdir = htmldir+'msds/'
    webmsdsdir = webhtmldir+'msds/'
    websafetyplansdir = webhtmldir+'Lab_Specific_Hygiene_Plans/'
    roomfile = home+'matinsy/etc/allrooms.dat'
    cheminfodir = htmldir+'cheminfo/'
    cheminfodata = home+'matinsy/cheminfodata/'
else:
    pass



tp = '<HTML>\n <HEAD><TITLE>SDS chemical inventory DB Problems </TITLE></HEAD>\n<BODY>\n'
hd = '<H1>'  
he = '</H1>\n'
lt = '<UL>'
le = '</UL>'
li = '<LI>'
dn = '</BODY>\n </HTML>\n'

print('********************************************')

bmsg = ' db cheminfo building beginning '
print(host,bmsg,btime)
print(cheminfodata)

********************************************
sausage  db cheminfo building beginning  2017-09-12 19:40
/home/huffman/work/matinsy/cheminfodata/


## definitions

In [79]:
def writehtml(ofile,htmltable,room):
    #room = df.room.unique()
    #room = room.tolist()[0]
    #print(getevaclink(room))
    #print(getchplink(room))
    #print(getsoplinks(room))

    #evaclink = getevaclink(room)
    #chplink = getchplink(room)
    #soplink = getsoplinks(room)
    #table = df.style.applymap(highlight_vals, subset=['regtype']).set_table_attributes("border=1").render()
    #nfpatable = roomdf.to_html(na_rep='0', col_space=12)##ask wes about this
    out = ' '.join(('<HTML>\n <HEAD><TITLE> chemical Information </TITLE> \
        </HEAD>\n<BODY>\n <H1> ',room,
        '<H2> CHEMINFO </H2>',htmltable,\
        '</BODY>\n </HTML>\n'))
    #tp = "<HTML>\n <HEAD><TITLE>SDS chemical inventory  </TITLE> \
    #    </HEAD>\n<BODY>\n                                           \
    #    <H1>Evaculation plans for this room </H1>                    \
    #    <H1> Hygene plans for this room </H1>                      \
    #    <H1> NFPA max scores for this room </H1>                    \
    #    
    #    <H1 style=\"color:red\" > Chemical Inventory  RED are Potentially Hazardous Substances</H1>\n    \
    #    \n"
    
    #dn = '</BODY>\n </HTML>\n'
    with open(ofile, 'w') as f:
        f.write(out)
        #f.write( tp)
        #f.write(df.style.applymap(highlight_vals, subset=['regtype']).set_table_attributes("border=1").render())
        #f.write(dn)
    #os.chmod(ofile, mod)
    return

def getbotsNroom(dbfile,room):
    conn = sqlite3.connect(dbfile)
    c = conn.cursor()
    rooms = []
    catid = []
    CAS = []
    d = {}
    c.execute('select catid,room,CAS,reorder,name from Bot where room=?',[room])
    tmp1 = c.fetchall()
    for i in range(len(tmp1)):
        catid.append(tmp1[i][0])
        rooms.append(tmp1[i][1])
        CAS.append(tmp1[i][2])
        tmp2 = []
        for j in range(1,len(tmp1[i])):
            tmp2.append(tmp1[i][j])
        d[tmp1[i][0]] = tmp2
        conn.commit()
    c.close()
    return d


def getallbots(dbfile):
    conn = sqlite3.connect(dbfile)
    c = conn.cursor()
    rooms = []
    catid = []
    CAS = []
    d = {}
    c.execute('select catid,room,CAS,reorder,name from Bot')
    tmp1 = c.fetchall()
    for i in range(len(tmp1)):
        catid.append(tmp1[i][0])
        rooms.append(tmp1[i][1])
        CAS.append(tmp1[i][2])
        tmp2 = []
        for j in range(1,len(tmp1[i])):
            tmp2.append(tmp1[i][j])
        d[tmp1[i][0]] = tmp2
        conn.commit()
    c.close()
    return d

#cas = '114460-21-8'
#cas = '107-07-3'
#  '0-0-0000',None
def ckcasvalidity(cas):
    #print('CAS',cas)
    exceptions = (';afjadjfakjf;ad','addad22222',None)#('114460-21-8', '1319-46-6',  '9003-70-7', '10486-00-7','9047-08-9','14024-63-6','28053-08-9','58-86-6','8005-03-6','9001-41-6','13520-83-7','13520-83-7','1250-23-4','9000-01-5','12030-88-5','67-09-5')
    if cas in exceptions:
        casflag = 2
        #print('CAS exception')
    elif '1-0-' in cas:
        casflag = 3
    else:
        #print(cas)
        cs = cas.split('-')
        #print('cs',cs)
        Nstr = ''.join(cs[:2])
        #print('Nstr',Nstr)
        Q = float(cs[-1])
        #print('Q',Q)
        LN = len(Nstr)
        Nlist = []
        for i,N in enumerate(Nstr):
            Nlist.append(float(N)*(LN-i))
        Nprod = np.mod(np.sum(Nlist),10)
        if Nprod == Q:
            casflag = 1
            #print('Valid CAS')
        else:
            casflag = 0
            #print('Bad CAS',cas,Nprod,Q)
    return casflag
#print(Q,Nprod,Nlist)
def casvalidlist(caslist):
    #TODO:  add wes cas filter
    validlist = []
    invalidlist = []
    exceptlist = []
    wescas = []
    for cas in caslist:
        flag = ckcasvalidity(cas)
        if flag == 1:
            validlist.append(cas)
        elif flag == 0:
            invalidlist.append(cas)
        elif flag == 2:
            exceptlist.append(cas)
        elif flag == 3:
            wescas.append(cas)
        else:
            pass
    return validlist,invalidlist,exceptlist,wescas

In [80]:
#test  = np.mod(7*6 + 6*6 + 5*4 + 3*6 +2*7 + 8*1,10)
#print(test)
caslist = list(set(df['CAS'].tolist()))
valid,invalid,exclist,wescas = casvalidlist(caslist)

In [81]:
print(len(valid),len(invalid),len(exclist),len(wescas))
print(invalid)


1479 24 1 58
['626-5-5-1', '109-0-4--6', '311-39-5', '24560-42-8', '15086-94-6', '231-85-3', '74-64-4', '17-1335-01', '67-09-5', '622-60-3', '67-66-1', '7740-43-9', '4996-61-3', '1250-23-4', '13446-34-6', '657-26-11', '7704-39-9', '1118-69-9', '8050-90-7', '630-45-2', '7311-3-0--0', '107-89-9', '29-49-92-0', '298-38-9']


In [35]:
cas = '114460-21-8'
cas = '1319-46-6'
cas = '10486-00-7'
cas = None
cas = '521-31-3'
cas = '7646-78-8'
casflag = ckcasvalidity(cas)
casflag

0

In [119]:
room = 'NS202'
d = getbotsNroom(dbfile,room)
df = pd.DataFrame.from_dict(d, orient='index')
df.rename(columns={0:'room',1:'CAS',2:'reorder',3:'name'},inplace=True)

In [84]:
for cas in invalid:
    print(df[df['CAS']== cas])

         room        CAS reorder             name
107229  NS202  626-5-5-1    1590  3-bromopyridine
         room         CAS reorder             name
107276  NS202  109-0-4--6    5564  2-bromopyridine
         room       CAS reorder          name
107296  NS202  311-39-5   00802  caffeic acid
         room         CAS reorder                                name
100648  NS202  24560-42-8   38781  2,2-dimethoxy-2-phenylacetophenone
         room         CAS reorder      name
104000  NS202  15086-94-6   45380  eosin y 
         room       CAS reorder                name
102646  NS202  231-85-3  222550  nile blue chloride
         room      CAS reorder           name
102357  NS202  74-64-4    9290  ethyl bromide
         room         CAS   reorder                  name
102499  NS202  17-1335-01  17133501  drystrip cover fluid
         room      CAS    reorder     name
106396  NS202  67-09-5  418001000  pinacol
         room       CAS reorder               name
101051  NS202  622-60-3    29

In [139]:
caslist = df['CAS'].tolist()
caslist = list(filter(None, set(caslist))) # fastest
ciddict = {}
for cas in caslist:
    print(cas)
    casflag = ckcasvalidity(cas)
    if casflag == 0:
        pass
    else:
        url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/'+cas+'/cids/json'
        response = urlopen(url)
        cid = json.loads(response.read())['IdentifierList']['CID']
        c = pcp.Compound.from_cid(cid)
        smiles = c.isomeric_smiles
        name = c.iupac_name
        mw = c.molecular_weight
        proplist = [cid,name,smiles,mw]
        ciddict[cas] = proplist

7540-51-4
CAS 7540-51-4
Bad CAS
119-53-9
CAS 119-53-9
Bad CAS
95-53-4
CAS 95-53-4
Bad CAS
1-0-1028
CAS 1-0-1028
Bad CAS
5743-04-4
CAS 5743-04-4
Bad CAS
50-32-8
CAS 50-32-8
Bad CAS
9006-59-1
CAS 9006-59-1
Bad CAS
1787-61-7
CAS 1787-61-7
Bad CAS
2016-42-4
CAS 2016-42-4
Valid CAS
624-49-7
CAS 624-49-7
Bad CAS
2747-05-9
CAS 2747-05-9
Bad CAS
88-72-2
CAS 88-72-2
Bad CAS
57-48-7
CAS 57-48-7
Bad CAS
1-0-1022
CAS 1-0-1022
Bad CAS
101-75-7
CAS 101-75-7
Bad CAS
1522-22-1
CAS 1522-22-1
Bad CAS
110-16-7
CAS 110-16-7
Bad CAS
130-22-3
CAS 130-22-3
Bad CAS
516-05-2
CAS 516-05-2
Bad CAS
137-07-5
CAS 137-07-5
Bad CAS
57-88-5
CAS 57-88-5
Bad CAS
3618-43-7
CAS 3618-43-7
Bad CAS
56-84-8
CAS 56-84-8
Bad CAS
7726-95-6
CAS 7726-95-6
Bad CAS
19902-08-0
CAS 19902-08-0
Bad CAS
581-64-6
CAS 581-64-6
Bad CAS
13446-53-2
CAS 13446-53-2
Bad CAS
7681-57-4
CAS 7681-57-4
Bad CAS
530-57-4
CAS 530-57-4
Bad CAS
120-82-1
CAS 120-82-1
Bad CAS
141-43-5
CAS 141-43-5
Bad CAS
7440-69-9
CAS 7440-69-9
Valid CAS
57-00-1
CAS 57-00-

In [141]:
dfstruct = pd.DataFrame.from_dict(ciddict,orient='index')
dfstruct.rename(columns={0:'cid',1:'name',2:'smiles',3:'mw'},inplace=True)
PandasTools.AddMoleculeColumnToFrame(dfstruct, smilesCol='smiles', molCol='struct', includeFingerprints=False)
htmltable = dfstruct.to_html()
ofile = cheminfodir+room+'.html'
writehtml(ofile,htmltable,room)

In [85]:
dd = getallbots(dbfile)
ddf = pd.DataFrame.from_dict(dd, orient='index')
ddf.rename(columns={0:'room',1:'CAS',2:'reorder',3:'name'},inplace=True)

In [90]:
caslist = list(filter(None,set(ddf['CAS'].tolist())))
caslist
#valid,invalid,exclist,wescas = casvalidlist(caslist)

['7646-78-8',
 '1079-66-9',
 '6485-79-6',
 '593-51-1',
 '20694-39-7',
 '4101-68-2',
 '71989-38-3',
 '15009-13-5',
 '52556-42-0',
 '7778-80-5',
 '3760-11-0',
 '122-51-0',
 '598-75-4',
 '122-39-4',
 '504-60-9',
 '1119-62-6',
 '123-19-3',
 '95-45-4',
 '501-65-5',
 '15244-36-7',
 '108321-42-2',
 '7488-55-3',
 '105-08-8',
 '6032-29-7',
 '7773-56-0',
 '509-34-2',
 '1071-23-4',
 '1310-65-2',
 '32701-19-2',
 '620-45-1',
 '5328-37-0',
 '98-59-9',
 '59-23-4',
 '67-56-1',
 '9005-46-3',
 '477-73-6',
 '7783-90-6',
 '2181-42-2',
 '109-66-0',
 '96-22-0',
 '544-92-3',
 '582-52-5',
 '28836-03-5',
 '1314-36-9',
 '606-68-8',
 '7791-18-6',
 '1-0-1033',
 '?8-83-4',
 '63451-35-4',
 '78-50-2',
 '1-0-1138',
 '90-80-2',
 '100-41-4',
 '5720-05-8',
 '1309-60-0',
 '142-45-0',
 '39455-18-0',
 '13455-24-8',
 '91079-40-2',
 '37199-22-7',
 '50-85-1',
 '9012-76-4',
 '79-09-4',
 '1122-81-2',
 '1-0-1164',
 '3248-05-3',
 '57-09-0',
 '50-70-4',
 '12650-88-3',
 '74-11-3',
 '107-31-3',
 '7787-20-4',
 '9001-75-6',
 '56-40-6'

In [10]:
store = HDFStore(cheminfodata+'store.h5')



In [ ]:
store['df'] = df  # save it
store['dfstruct'] = dfstruct
store['ddf'] = ddf
#df = store['df']  # load it

In [11]:
df = store['df']

In [12]:
df

,room,CAS,reorder,name
100001,NS202,1-0-0001,blank,blank
100100,NS202,514-10-3,a9424,abietic acid
100101,NS202,9000-01-5,g85,acacia powder
100102,NS202,60-35-5,a4250,acetamide
100104,NS202,1071-73-4,a20804,3-acetyl-1-propanol
100105,NS202,98-86-2,102410010,acetophenone
100106,NS202,98-86-2,102410010,acetophenone
100107,NS202,98-86-2,102410010,acetophenone
100108,NS202,103-84-4,None,acetanilide
100109,NS202,50-78-2,158185000,acetylsalicylic acid
